In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "a39217e9-0d31-488c-9be3-8edc3315717a",
   "metadata": {},
   "source": [
    "# Solution 1/3: Tracking by detection and simple frame-by-frame matching\n",
    "\n",
    "You could also run this notebook on your laptop, a GPU is not needed."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7b44ba16-6ace-46c0-97b9-51b1c21b1500",
   "metadata": {},
   "source": [
    "This notebook was written by Benjamin Gallusser."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "fee04091-df5a-43f2-bed4-a8643b44127b",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Import packages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "22bdc7a8",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Force keras to run on CPU\n",
    "import os\n",
    "os.environ[\"CUDA_VISIBLE_DEVICES\"] = \"\"\n",
    "\n",
    "# Notebook at full width in the browser\n",
    "from IPython.display import display, HTML\n",
    "display(HTML(\"<style>.container { width:100% !important; }</style>\"))\n",
    "\n",
    "import sys\n",
    "from urllib.request import urlretrieve\n",
    "from pathlib import Path\n",
    "from collections import defaultdict\n",
    "from abc import ABC, abstractmethod\n",
    "\n",
    "import matplotlib\n",
    "import matplotlib.pyplot as plt\n",
    "%matplotlib inline\n",
    "matplotlib.rcParams[\"image.interpolation\"] = \"none\"\n",
    "matplotlib.rcParams['figure.figsize'] = (14, 10)\n",
    "import numpy as np\n",
    "from tifffile import imread, imwrite\n",
    "from tqdm.auto import tqdm\n",
    "import skimage\n",
    "import pandas as pd\n",
    "import scipy\n",
    "\n",
    "from stardist import fill_label_holes, random_label_cmap\n",
    "from stardist.plot import render_label\n",
    "from stardist.models import StarDist2D\n",
    "from stardist import _draw_polygons\n",
    "from csbdeep.utils import normalize\n",
    "\n",
    "import napari\n",
    "\n",
    "lbl_cmap = random_label_cmap()\n",
    "# Pretty tqdm progress bars \n",
    "! jupyter nbextension enable --py widgetsnbextension"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8a0c88d0",
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_img_label(img, lbl, img_title=\"image\", lbl_title=\"label\", **kwargs):\n",
    "    fig, (ai,al) = plt.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1)))\n",
    "    im = ai.imshow(img, cmap='gray', clim=(0,1))\n",
    "    ai.set_title(img_title)\n",
    "    ai.axis(\"off\")\n",
    "    al.imshow(render_label(lbl, img=.3*img, normalize_img=False, cmap=lbl_cmap))\n",
    "    al.set_title(lbl_title)\n",
    "    al.axis(\"off\")\n",
    "    plt.tight_layout()\n",
    "    \n",
    "def preprocess(X, Y, axis_norm=(0,1)):\n",
    "    # normalize channels independently\n",
    "    X = np.stack([normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X, leave=True, desc=\"Normalize images\")])\n",
    "    # fill holes in labels\n",
    "    Y = np.stack([fill_label_holes(y) for y in tqdm(Y, leave=True, desc=\"Fill holes in labels\")])\n",
    "    return X, Y"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "18392fbe-a00e-49f0-a6c9-1f9e122af817",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Inspect the dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d2e95c3b-5bb3-4240-a4c1-63c2f898bdb8",
   "metadata": {},
   "outputs": [],
   "source": [
    "base_path = Path(\"data/exercise1\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ff9b2f5f-188b-4337-8aad-c2ba7af43495",
   "metadata": {},
   "outputs": [],
   "source": [
    "x = np.stack([imread(xi) for xi in sorted((base_path / \"images\").glob(\"*.tif\"))])  # images\n",
    "y = np.stack([imread(yi) for yi in sorted((base_path / \"gt_tracking\").glob(\"*.tif\"))])  # ground truth annotations\n",
    "assert x.shape == y.shape\n",
    "print(f\"Number of images: {len(x)}\")\n",
    "print(f\"Shape of images: {x[0].shape}\")\n",
    "x, y = preprocess(x, y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e4ed1f21-4952-4785-9be0-20b6956439bf",
   "metadata": {},
   "outputs": [],
   "source": [
    "idx = 0\n",
    "plot_img_label(x[idx], y[idx])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "caa5bbf2-95e0-4325-9d84-1ccb76ad4164",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x, name=\"image\");"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1c6e84e8-c070-4e10-ad7d-313477e9f38d",
   "metadata": {},
   "source": [
    "<div class=\"alert alert-block alert-danger\"><h3>Napari in a jupyter notebook:</h3>\n",
    "    \n",
    "- To have napari working in a jupyter notebook, you need to use up-to-date versions of napari, pyqt and pyqt5, as is the case in the conda environments provided together with this exercise.\n",
    "- When you are coding and debugging, close the napari viewer with `viewer.close()` to avoid problems with the two event loops of napari and jupyter.\n",
    "- **If a cell is not executed (empty square brackets on the left of a cell) despite you running it, running it a second time right after will usually work.**\n",
    "</div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6fcd7fd2-6abe-412f-9fa5-ceaa243622a2",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer.add_labels(y, name=\"labels\");"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c2a6fc09-5d93-494f-91e4-c0673488b7b9",
   "metadata": {},
   "outputs": [],
   "source": [
    "links = np.loadtxt(base_path / \"gt_tracking\" / \"man_track.txt\", dtype=int)\n",
    "links = pd.DataFrame(data=links, columns=[\"track_id\", \"from\", \"to\", \"parent_id\"])\n",
    "print(\"Links\")\n",
    "links[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f8e8e050-2be9-4248-a110-251004f09ede",
   "metadata": {},
   "outputs": [],
   "source": [
    "def visualize_tracks(viewer, y, links=None, name=\"\"):\n",
    "    \"\"\"Utility function to visualize segmentation and tracks\"\"\"\n",
    "    max_label = max(links.max(), y.max()) if links is not None else y.max()\n",
    "    colorperm = np.random.default_rng(42).permutation((np.arange(1, max_label + 2)))\n",
    "    tracks = []\n",
    "    for t, frame in enumerate(y):\n",
    "        centers = skimage.measure.regionprops(frame)\n",
    "        for c in centers:\n",
    "            tracks.append([colorperm[c.label], t, int(c.centroid[0]), int(c.centroid[1])])\n",
    "    tracks = np.array(tracks)\n",
    "    tracks = tracks[tracks[:, 0].argsort()]\n",
    "    \n",
    "    graph = {}\n",
    "    if links is not None:\n",
    "        divisions = links[links[:,3] != 0]\n",
    "        for d in divisions:\n",
    "            if colorperm[d[0]] not in tracks[:, 0] or colorperm[d[3]] not in tracks[:, 0]:\n",
    "                continue\n",
    "            graph[colorperm[d[0]]] = [colorperm[d[3]]]\n",
    "\n",
    "    viewer.add_labels(y, name=f\"{name}_detections\")\n",
    "    viewer.layers[f\"{name}_detections\"].contour = 3\n",
    "    viewer.add_tracks(tracks, name=f\"{name}_tracks\", graph=graph)\n",
    "    return tracks"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "48ab6da7-264d-47d5-8b9a-a13ae3d32c1e",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "visualize_tracks(viewer, y, links.to_numpy(), \"ground_truth\");"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4d6df6e2-332f-497c-a007-f1e126ffe325",
   "metadata": {
    "incorrectly_encoded_metadata": "jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.1\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.1: Highlight the cell divisions</h3>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "74bddab1-5244-4012-bf4b-32af4b7b9d39",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution Exercise 1.1\n",
    "def extract_divisions(y, links):\n",
    "    \"\"\"Utility function to extract divisions\"\"\"    \n",
    "    daughters = links[links[:,3] != 0]\n",
    "    divisions = np.zeros_like(y)\n",
    "\n",
    "    for d in daughters:\n",
    "        if d[0] not in y or d[3] not in y:\n",
    "            continue\n",
    "        divisions[d[1]][y[d[1]] == d[0]] = d[0]\n",
    " \n",
    "    return divisions"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "03e59dc4-d89e-4916-b6b0-8f9ce4f6b1c6",
   "metadata": {},
   "source": [
    "Feel free to test your function with this minimal example (with toy \"images\" in 1D)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "3afb105c-2078-472f-bf76-739a5566bd6b",
   "metadata": {},
   "outputs": [],
   "source": [
    "def test_extract_divisions():\n",
    "    y = np.array([[0, 10, 0, 0], [0, 11, 12, 13], [0, 11, 12, 13], [0, 11, 0, 13]])\n",
    "    links = pd.DataFrame([[11, 1, 2, 10], [12, 1, 3, 10]], columns=[\"track_id\", \"from\", \"to\", \"parent_id\"])\n",
    "    divs = extract_divisions(y, links.to_numpy())\n",
    "    expected_divs = np.array([[0, 0, 0, 0], [0, 11, 12, 0], [0, 0, 0, 0], [0, 0, 0, 0]])\n",
    "    if np.all(divs == expected_divs):\n",
    "        print(\"Success :)\")\n",
    "    else:\n",
    "        print(f\"Output {divs} does not match expected output\\n{expected_divs}\")\n",
    "\n",
    "test_extract_divisions()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "effa4ff3-4308-4331-9964-7b0e01069433",
   "metadata": {},
   "source": [
    "Visualize the output of your function:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "359d586a-af96-4d80-8941-d56b03220211",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "divisions = extract_divisions(y, links.to_numpy())\n",
    "viewer.add_labels(divisions, name=\"divisions\");"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "835ca5c3-96b9-4229-81b4-37669bc3ba11",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Object detection using a pre-trained neural network"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a8f156b6-2add-4974-ba6f-a4f813b26d50",
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "idx = 0\n",
    "model = StarDist2D(None, name=\"stardist_breast_cancer\", basedir=\"models\")\n",
    "(detections, details), (prob, _) = model.predict_instances(x[idx], scale=(1, 1), return_predict=True)\n",
    "plot_img_label(x[idx], detections, lbl_title=\"detections\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "fb767441-bab9-426f-83e7-5c6b7d094ad3",
   "metadata": {},
   "outputs": [],
   "source": [
    "coord, points, polygon_prob = details['coord'], details['points'], details['prob']\n",
    "plt.figure(figsize=(24,12))\n",
    "plt.subplot(121)\n",
    "plt.title(\"Predicted Polygons\")\n",
    "_draw_polygons(coord, points, polygon_prob, show_dist=True)\n",
    "plt.imshow(x[idx], cmap='gray'); plt.axis('off')\n",
    "\n",
    "plt.subplot(122)\n",
    "plt.title(\"Object center probability\")\n",
    "plt.imshow(prob, cmap='magma'); plt.axis('off')\n",
    "plt.tight_layout()\n",
    "plt.show() "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b06258ea-2cbe-43c6-84b9-e6e45e08082d",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.2\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.2: Explore the parameters of cell detection</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4bc2e533-58b9-4b5b-9c61-adb9cdd8a245",
   "metadata": {},
   "outputs": [],
   "source": [
    "scale = (1.0, 1.0)\n",
    "pred = [model.predict_instances(xi, show_tile_progress=False, scale=scale)\n",
    "              for xi in tqdm(x)]\n",
    "detections = [xi[0] for xi in pred]\n",
    "detections = np.stack([skimage.segmentation.relabel_sequential(d)[0] for d in detections])  # ensure that label ids are contiguous and start at 1 for each frame \n",
    "centers = [xi[1][\"points\"] for xi in pred]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f997dd09-3121-402b-b7b9-9e5498bad2a4",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "viewer.add_labels(detections, name=f\"detections_scale_{scale}\");"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ff23798d-57d7-4a18-a00c-9b7d5982e235",
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10,6))\n",
    "plt.bar(range(len(centers)), [len(xi) for xi in centers])\n",
    "plt.title(f\"Number of detections in each frame (scale={scale})\")\n",
    "plt.xticks(range(len(centers)))\n",
    "plt.show();"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "3c033125-d4f5-48bb-9b2c-47aa7c8e5ca7",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Checkpoint 1\n",
    "<div class=\"alert alert-block alert-success\"><h3>Checkpoint 1: We have good detections, now on to the linking.</h3></div>"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7b9f5a9a-dfe4-4568-a85d-a1dfc3eba19a",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true",
    "tags": []
   },
   "source": [
    "## Greedy linking by nearest neighbor"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cbc3769f-6135-4b42-a779-610353fa7cc1",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.3\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.3: Write a function that computes pairwise euclidian distances given two lists of points</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dd881fa8-2b05-4cd6-b7d1-921ca66319ac",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution Exercise 1.3\n",
    "\n",
    "def pairwise_euclidian_distance(points0, points1):\n",
    "    print(\"Iterative pairwise euclidian distance\")\n",
    "    dists = []\n",
    "    for p0 in points0:\n",
    "        for p1 in points1:\n",
    "            dists.append(np.sqrt(((p0 - p1)**2).sum()))\n",
    "            \n",
    "    dists = np.array(dists).reshape(len(points0), len(points1))\n",
    "    return dists\n",
    "\n",
    "# def pairwise_euclidian_distance(points0, points1):\n",
    "#     # Numpy-based, but still slow\n",
    "#     print(\"Vectorized pairwise euclidian distance\")\n",
    "#     return np.apply_along_axis(\n",
    "#         np.linalg.norm,\n",
    "#         2,\n",
    "#         points0[:, None, :] - points1[None, :, :]\n",
    "#     )\n",
    "\n",
    "# def pairwise_euclidian_distance(points0, points1):\n",
    "#     print(\"Scipy pairwise euclidian distance\")\n",
    "#     return scipy.spatial.distance.cdist(points0, points1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9ec05e1f-436d-4309-9cd4-1f47a30ce189",
   "metadata": {},
   "outputs": [],
   "source": [
    "green_points = np.load(\"points.npz\")[\"green\"]\n",
    "cyan_points = np.load(\"points.npz\")[\"cyan\"]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ec69da1e-cab4-473c-85c1-c45fb1e2ba16",
   "metadata": {},
   "outputs": [],
   "source": [
    "%time dists = pairwise_euclidian_distance(green_points, cyan_points)\n",
    "assert np.allclose(dists, np.load(\"points.npz\")[\"dists_green_cyan\"])"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0c8a72df-3715-4a24-85a9-1f87a86e85ef",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.4\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.4: Write a function that greedily extracts a nearest neighbors assignment given a cost matrix</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2ff42dde-f881-46e8-a9ed-1271e467580c",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution exercise 1.4\n",
    "\n",
    "def nearest_neighbor(cost_matrix, threshold=np.finfo(float).max):\n",
    "    \"\"\"Greedy nearest neighbor assignment.\n",
    "    \n",
    "    Each point in both sets can only be assigned once. \n",
    "    \n",
    "    Args:\n",
    "\n",
    "        cost_matrix: m x n matrix with pairwise linking costs of two sets of points.\n",
    "\n",
    "    Returns:\n",
    "\n",
    "        Tuple of lists (ids frame t, ids frame t+1).\n",
    "    \"\"\"\n",
    "    A = cost_matrix.copy().astype(float)\n",
    "    ids_from = []\n",
    "    ids_to = []\n",
    "    for i in range(min(A.shape[0], A.shape[1])):\n",
    "        row, col = np.unravel_index(A.argmin(), A.shape)\n",
    "        \n",
    "        if A.min() >= threshold:\n",
    "                break\n",
    "        ids_from.append(row)\n",
    "        ids_to.append(col)\n",
    "        A[row, :] = cost_matrix.max() + 1\n",
    "        A[:, col] = cost_matrix.max() + 1\n",
    "\n",
    "    return np.array(ids_from), np.array(ids_to)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "888d1056-6502-4f55-9c6c-237308faf14b",
   "metadata": {},
   "outputs": [],
   "source": [
    "test_matrix = np.array([\n",
    "    [8, 2, 8],\n",
    "    [9, 9, 9],\n",
    "    [1, 8, 8],\n",
    "    [8, 3, 8],\n",
    "])\n",
    "idx_from, idx_to = nearest_neighbor(test_matrix, threshold=8)\n",
    "assert np.all(idx_from == [2, 0])\n",
    "assert np.all(idx_to == [0, 1])"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bca8c7dc-e44e-4b5c-b534-b4a4ef3130fe",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.5\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.5: Complete a thresholded nearest neighbor linker using your functions from exercises 1.3 and 1.4</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d96c279f-5e0f-47f5-b35f-4d275a486161",
   "metadata": {},
   "outputs": [],
   "source": [
    "class FrameByFrameLinker(ABC):\n",
    "    \"\"\"Abstract base class for linking detections by considering pairs of adjacent frames.\"\"\"\n",
    "    \n",
    "    def link(self, detections, images=None):\n",
    "        \"\"\"Links detections in t frames.\n",
    "        \n",
    "        Args:\n",
    "        \n",
    "            detections:\n",
    "            \n",
    "                List of t numpy arrays of shape (x,y) with contiguous label ids. Background = 0.\n",
    "                \n",
    "            images (optional):\n",
    "            \n",
    "                List of t numpy arrays of shape (x,y).\n",
    "        \n",
    "        Returns:\n",
    "        \n",
    "            Linking dictionary:\n",
    "                \"links\":\n",
    "                    \n",
    "                    Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: \n",
    "                    - idgs_from: [from0, from1 , ...])\n",
    "                    - ids_to: [to0, to1 , ...])\n",
    "                \n",
    "                \"births\": List of ids from frame t that are \n",
    "                \"deaths\": List of ids.\n",
    "            Ids are one-based, 0 is reserved for background.\n",
    "        \"\"\"\n",
    "        if images is not None:\n",
    "            assert len(images) == len(detections)\n",
    "        else:\n",
    "            images = [None] * len(detections)\n",
    "\n",
    "        links = []\n",
    "        for i in tqdm(range(len(images) - 1), desc=\"Linking\"):\n",
    "            detections0 = detections[i]\n",
    "            detections1 = detections[i+1]\n",
    "            self._assert_relabeled(detections0)\n",
    "            self._assert_relabeled(detections1)\n",
    "            \n",
    "            cost_matrix = self.linking_cost_function(detections0, detections1, images[i], images[i+1])\n",
    "            li = self._link_two_frames(cost_matrix)\n",
    "            self._assert_links(links=li, time=i, detections0=detections0, detections1=detections1) \n",
    "            links.append(li)\n",
    "            \n",
    "        return links\n",
    "\n",
    "    @abstractmethod\n",
    "    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):\n",
    "        \"\"\"Calculate features for each detection and extract pairwise costs.\n",
    "        \n",
    "        To be overwritten in subclass.\n",
    "        \n",
    "        Args:\n",
    "        \n",
    "            detections0: image with background 0 and detections 1, ..., m\n",
    "            detections1: image with backgruond 0 and detections 1, ..., n\n",
    "            image0 (optional): image corresponding to detections0\n",
    "            image1 (optional): image corresponding to detections1\n",
    "            \n",
    "        Returns:\n",
    "        \n",
    "            m x n cost matrix \n",
    "        \"\"\"\n",
    "        pass\n",
    "    \n",
    "    @abstractmethod\n",
    "    def _link_two_frames(self, cost_matrix):\n",
    "        \"\"\"Link two frames.\n",
    "        \n",
    "        To be overwritten in subclass.\n",
    "\n",
    "        Args:\n",
    "\n",
    "            cost_matrix: m x n matrix\n",
    "\n",
    "        Returns:\n",
    "        \n",
    "            \"links\":\n",
    "\n",
    "                Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: \n",
    "                - idgs_from: [from0, from1 , ...])\n",
    "                - ids_to: [to0, to1 , ...])\n",
    "\n",
    "            \"births\": List of ids from frame t that are \n",
    "            \"deaths\": List of ids.\n",
    "            \n",
    "            Ids are one-based, 0 is reserved for background.\n",
    "        \"\"\"\n",
    "        pass\n",
    "\n",
    "    def relabel_detections(self, detections, links):\n",
    "        \"\"\"Relabel dense detections according to computed links, births and deaths.\n",
    "        \n",
    "        Args:\n",
    "        \n",
    "            detections: \n",
    "                 \n",
    "                 List of t numpy arrays of shape (x,y) with contiguous label ids. Background = 0.\n",
    "                 \n",
    "            links:\n",
    "                \n",
    "                List of t linking dictionaries, each containing:\n",
    "                    \"links\": Tuple of lists (ids frame t, ids frame t+1),\n",
    "                    \"births\": List of ids,\n",
    "                    \"deaths\": List of ids.\n",
    "                Ids are one-based, 0 is reserved for background.\n",
    "        \"\"\"\n",
    "        detections = detections.copy()\n",
    "        \n",
    "        assert len(detections) - 1 == len(links)\n",
    "        self._assert_relabeled(detections[0])\n",
    "        out = [detections[0]]\n",
    "        n_tracks = out[0].max()\n",
    "        lookup_tables = [{i: i for i in range(1, out[0].max() + 1)}]\n",
    "\n",
    "        for i in tqdm(range(len(links)), desc=\"Recoloring detections\"):\n",
    "            (ids_from, ids_to) = links[i][\"links\"]\n",
    "            births = links[i][\"births\"]\n",
    "            deaths = links[i+1][\"deaths\"] if i+1 < len(links) else []\n",
    "            new_frame = np.zeros_like(detections[i+1])\n",
    "            self._assert_relabeled(detections[i+1])\n",
    "            \n",
    "            lut = {}\n",
    "            for _from, _to in zip(ids_from, ids_to):\n",
    "                # Copy over ID\n",
    "                new_frame[detections[i+1] == _to] = lookup_tables[i][_from]\n",
    "                lut[_to] = lookup_tables[i][_from]\n",
    "\n",
    "            \n",
    "            # Start new track for birth tracks\n",
    "            for b in births:\n",
    "                if b in deaths:\n",
    "                    continue\n",
    "                \n",
    "                n_tracks += 1\n",
    "                lut[b] = n_tracks\n",
    "                new_frame[detections[i+1] == b] = n_tracks\n",
    "                \n",
    "            # print(lut)\n",
    "            lookup_tables.append(lut)\n",
    "            out.append(new_frame)\n",
    "                \n",
    "        return np.stack(out)\n",
    "\n",
    "    def _assert_links(self, links, time, detections0, detections1):\n",
    "        if len(links[\"links\"][0]) != len(links[\"links\"][1]):\n",
    "            raise RuntimeError(\"Format of links['links'] not correct.\")\n",
    "            \n",
    "        if sorted([*links[\"links\"][0], *links[\"deaths\"]]) != list(range(1, len(np.unique(detections0)))):\n",
    "            raise RuntimeError(f\"Some detections in frame {time} are not properly assigned as either linked or death.\")\n",
    "            \n",
    "        if sorted([*links[\"links\"][1], *links[\"births\"]]) != list(range(1, len(np.unique(detections1)))):\n",
    "            raise RuntimeError(f\"Some detections in frame {time + 1} are not properly assigned as either linked or birth.\")\n",
    "            \n",
    "        for b in links[\"births\"]:\n",
    "            if b in links[\"links\"][1]:\n",
    "                raise RuntimeError(f\"Links frame {time+1}: Detection {b} marked as birth, but also linked.\")\n",
    "        \n",
    "        for d in links[\"deaths\"]:\n",
    "            if d in links[\"links\"][0]:\n",
    "                raise RuntimeError(f\"Links frame {time}: Detection {d} marked as death, but also linked.\")\n",
    "        \n",
    "        \n",
    "    def _assert_relabeled(self, x):\n",
    "        if x.min() < 0:\n",
    "            raise ValueError(\"Negative ID in detections.\")\n",
    "        if x.min() == 0:\n",
    "            n = x.max() + 1\n",
    "        else:\n",
    "            n = x.max()\n",
    "        if n != len(np.unique(x)):\n",
    "            raise ValueError(\"Detection IDs are not contiguous.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8e593e4e-8d90-4320-b427-57ac1dfeeb14",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution Exercise 1.5\n",
    "\n",
    "class NearestNeighborLinkerEuclidian(FrameByFrameLinker):\n",
    "    \"\"\".\n",
    "    \n",
    "    Args:\n",
    "    \n",
    "        threshold (float): Maximum euclidian distance for linking.\n",
    "    \"\"\"\n",
    "    \n",
    "    def __init__(self, threshold=np.finfo(float).max, *args, **kwargs):\n",
    "        self.threshold = threshold\n",
    "        super().__init__(*args, **kwargs)\n",
    "    \n",
    "    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):\n",
    "        \"\"\" Get centroids from detections and compute pairwise euclidian distances.\n",
    "                \n",
    "        Args:\n",
    "        \n",
    "            detections0: image with background 0 and detections 1, ..., m\n",
    "            detections1: image with backgruond 0 and detections 1, ..., n\n",
    "            \n",
    "        Returns:\n",
    "        \n",
    "            m x n cost matrix \n",
    "        \"\"\"\n",
    "        # regionprops regions are sorted by label\n",
    "        regions0 = skimage.measure.regionprops(detections0)\n",
    "        points0 = [np.array(r.centroid) for r in regions0]\n",
    "        \n",
    "        regions1 = skimage.measure.regionprops(detections1)\n",
    "        points1 = [np.array(r.centroid) for r in regions1]\n",
    "        \n",
    "        dists = []\n",
    "        for p0 in points0:\n",
    "            for p1 in points1:\n",
    "                dists.append(np.sqrt(((p0 - p1)**2).sum()))\n",
    "\n",
    "        dists = np.array(dists).reshape(len(points0), len(points1))\n",
    "        \n",
    "        return dists\n",
    "    \n",
    "    def _link_two_frames(self, cost_matrix):\n",
    "        \"\"\"Greedy nearest neighbor assignment.\n",
    "\n",
    "        Each point in both sets can only be assigned once. \n",
    "\n",
    "        Args:\n",
    "\n",
    "            cost_matrix: m x n matrix containing pairwise linking costs of two sets of points.\n",
    "\n",
    "        Returns:\n",
    "            \"links\":\n",
    "\n",
    "                Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: \n",
    "                - idgs_from: [from0, from1 , ...])\n",
    "                - ids_to: [to0, to1 , ...])\n",
    "\n",
    "            \"births\": List of ids from frame t that are \n",
    "            \"deaths\": List of ids.\n",
    "            \n",
    "            Ids are one-based, 0 is reserved for background.\n",
    "        \"\"\"\n",
    "        ids_from, ids_to = nearest_neighbor(cost_matrix, self.threshold)\n",
    "        births = np.array(list(set(range(cost_matrix.shape[1])) - set(ids_to)))\n",
    "        deaths = np.array(list(set(range(cost_matrix.shape[0])) - set(ids_from)))\n",
    "        \n",
    "        # Account for +1 offset of the dense labels\n",
    "        ids_from += 1\n",
    "        ids_to += 1\n",
    "        births += 1\n",
    "        deaths += 1\n",
    "        \n",
    "        links = {\"links\": (ids_from, ids_to), \"births\": births, \"deaths\": deaths}\n",
    "        return links"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d005234b-9c4b-4c60-8656-00d0a92790fb",
   "metadata": {},
   "outputs": [],
   "source": [
    "# nn_linker = NearestNeighborLinkerEuclidian(threshold=1000) # Explore different values of `threshold`\n",
    "nn_linker = NearestNeighborLinkerEuclidian(threshold=50) # Solution param\n",
    "nn_links = nn_linker.link(detections)\n",
    "nn_tracks = nn_linker.relabel_detections(detections, nn_links)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "76932eb8-0186-4777-8818-8e976940f965",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "visualize_tracks(viewer, nn_tracks, name=\"nn\");"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b5ab42b7-ef07-4476-bc7d-d329ea91636b",
   "metadata": {
    "tags": []
   },
   "source": [
    "## Checkpoint 2\n",
    "<div class=\"alert alert-block alert-success\"><h3>Checkpoint 2: We built a basic tracking algorithm from scratch :).</h3></div>"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f241c4fa-e205-4de4-b9c6-7c0237459f9d",
   "metadata": {
    "incorrectly_encoded_metadata": "tags=[] jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.6\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.6: Estimate the global drift of the data</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bb557dc8-9ffa-4425-acfc-1c0f8f7d587c",
   "metadata": {},
   "outputs": [],
   "source": [
    "class NearestNeighborLinkerDriftCorrection(NearestNeighborLinkerEuclidian):\n",
    "    \"\"\".\n",
    "    \n",
    "    Args:\n",
    "        \n",
    "        drift: tuple of (x,y) drift correction per frame.\n",
    "    \"\"\"\n",
    "    \n",
    "    def __init__(self, drift, *args, **kwargs):\n",
    "        self.drift = np.array(drift)\n",
    "        super().__init__(*args, **kwargs)\n",
    "    \n",
    "    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):\n",
    "        \"\"\" Get centroids from detections and compute pairwise euclidian distances with drift correction.\n",
    "                \n",
    "        Args:\n",
    "        \n",
    "            detections0: image with background 0 and detections 1, ..., m\n",
    "            detections1: image with backgruond 0 and detections 1, ..., n\n",
    "            \n",
    "        Returns:\n",
    "        \n",
    "            m x n cost matrix \n",
    "        \"\"\"\n",
    "        # regionprops regions are sorted by label\n",
    "        regions0 = skimage.measure.regionprops(detections0)\n",
    "        points0 = [np.array(r.centroid) for r in regions0]\n",
    "        \n",
    "        regions1 = skimage.measure.regionprops(detections1)\n",
    "        points1 = [np.array(r.centroid) for r in regions1]\n",
    "        \n",
    "        dists = []\n",
    "        for p0 in points0:\n",
    "            for p1 in points1:\n",
    "                dists.append(np.sqrt(((p0 + self.drift - p1)**2).sum()))\n",
    "\n",
    "        dists = np.array(dists).reshape(len(points0), len(points1))\n",
    "        \n",
    "        return dists"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4a706de6-da8f-4503-870b-86fd4aca123b",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution Exercise 1.6\n",
    "drift_linker = NearestNeighborLinkerDriftCorrection(threshold=50, drift=(-20, 0)) # SOLUTION params\n",
    "drift_links = drift_linker.link(detections)\n",
    "drift_tracks = drift_linker.relabel_detections(detections, drift_links)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "fbf2c9e3-6dbc-4f4a-b66b-44f7852396ee",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "visualize_tracks(viewer, drift_tracks, name=\"drift\");"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8c96e9e0-0fcc-4f2b-b635-3797df6e8ff9",
   "metadata": {
    "tags": []
   },
   "source": [
    "## Optimal frame-by-frame matching (*Linear assignment problem* or *Weighted bipartite matching*)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0c73cb9b-81c2-4d7b-8a53-3ab2497cb995",
   "metadata": {
    "incorrectly_encoded_metadata": "jp-MarkdownHeadingCollapsed=true tags=[] jp-MarkdownHeadingCollapsed=true"
   },
   "source": [
    "## Exercise 1.7\n",
    "<div class=\"alert alert-block alert-info\"><h3>Exercise 1.7: Perform optimal frame-by-frame linking</h3></div>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "20007cf3-85b2-4025-a9de-77616d08ec92",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Solution exercise 1.7\n",
    "\n",
    "class BipartiteMatchingLinker(FrameByFrameLinker):\n",
    "    \"\"\".\n",
    "    \n",
    "    Args:\n",
    "        threshold (float): Maximum euclidian distance for linking.\n",
    "        drift: tuple of (x,y) drift correction per frame.\n",
    "        birth_cost_factor (float): Multiply factor with maximum entry in cost matrix.\n",
    "        death_cost_factor (float): Multiply factor with maximum entry in cost matrix.\n",
    "    \"\"\"\n",
    "    \n",
    "    def __init__(\n",
    "        self,\n",
    "        threshold=np.finfo(float).max,\n",
    "        drift=(0,0),\n",
    "        birth_cost_factor=1.05,\n",
    "        death_cost_factor=1.05,\n",
    "        *args,\n",
    "        **kwargs\n",
    "    ):\n",
    "        self.threshold = threshold\n",
    "        self.drift = np.array(drift)\n",
    "        self.birth_cost_factor = birth_cost_factor\n",
    "        self.death_cost_factor = death_cost_factor\n",
    "        \n",
    "        super().__init__(*args, **kwargs)\n",
    "        \n",
    "    def linking_cost_function(self, detections0, detections1, image0=None, image1=None):\n",
    "        \"\"\" Get centroids from detections and compute pairwise euclidian distances with drift correction.\n",
    "                \n",
    "        Args:\n",
    "        \n",
    "            detections0: image with background 0 and detections 1, ..., m\n",
    "            detections1: image with backgruond 0 and detections 1, ..., n\n",
    "            \n",
    "        Returns:\n",
    "        \n",
    "            m x n cost matrix \n",
    "        \"\"\"\n",
    "        # regionprops regions are sorted by label\n",
    "        regions0 = skimage.measure.regionprops(detections0)\n",
    "        points0 = [np.array(r.centroid) for r in regions0]\n",
    "        \n",
    "        regions1 = skimage.measure.regionprops(detections1)\n",
    "        points1 = [np.array(r.centroid) for r in regions1]\n",
    "        \n",
    "        dists = []\n",
    "        for p0 in points0:\n",
    "            for p1 in points1:\n",
    "                dists.append(np.sqrt(((p0 + self.drift - p1)**2).sum()))\n",
    "\n",
    "        dists = np.array(dists).reshape(len(points0), len(points1))\n",
    "        \n",
    "        return dists\n",
    "    \n",
    "    def _link_two_frames(self, cost_matrix):\n",
    "        \"\"\"Weighted bipartite matching with square matrix from Jaqaman et al (2008).\n",
    "\n",
    "        Args:\n",
    "\n",
    "            cost_matrix: m x n matrix.\n",
    "\n",
    "        Returns:\n",
    "            \"links\":\n",
    "    \n",
    "                Tuple of lists. Links from frame t to frame t+1 of form (from0, to0) are split up into two lists: \n",
    "                    - idgs_from: [from0, from1 , ...])\n",
    "                    - ids_to: [to0, to1 , ...])\n",
    "                \n",
    "                \"births\": List of ids from frame t that are \n",
    "                \"deaths\": List of ids.\n",
    "                \n",
    "            Ids are one-based, 0 is reserved for background.\n",
    "        \"\"\"\n",
    "        \n",
    "        cost_matrix = cost_matrix.copy().astype(float)\n",
    "        b = self.birth_cost_factor * min(self.threshold, cost_matrix.max())\n",
    "        d = self.death_cost_factor * min(self.threshold, cost_matrix.max())\n",
    "        no_link = max(cost_matrix.max(), max(b, d)) * 1e9\n",
    "        \n",
    "        cost_matrix[cost_matrix > self.threshold] = no_link\n",
    "        lower_right = cost_matrix.transpose()\n",
    "\n",
    "        deaths = np.full(shape=(cost_matrix.shape[0], cost_matrix.shape[0]), fill_value=no_link)\n",
    "        np.fill_diagonal(deaths, d)\n",
    "        births = np.full(shape=(cost_matrix.shape[1], cost_matrix.shape[1]), fill_value=no_link)\n",
    "        np.fill_diagonal(births, b)\n",
    "        \n",
    "        square_cost_matrix = np.block([\n",
    "            [cost_matrix, deaths],\n",
    "            [births, lower_right],\n",
    "        ])\n",
    "        row_ind, col_ind = scipy.optimize.linear_sum_assignment(square_cost_matrix)\n",
    "        \n",
    "        ids_from = []\n",
    "        ids_to = []\n",
    "        births = []\n",
    "        deaths = []\n",
    "        for row, col in zip(row_ind, col_ind):\n",
    "            if row < cost_matrix.shape[0] and col < cost_matrix.shape[1]:\n",
    "                ids_from.append(row)\n",
    "                ids_to.append(col)\n",
    "\n",
    "            if row >= cost_matrix.shape[0] and col < cost_matrix.shape[1]:\n",
    "                births.append(col)\n",
    "            if row < cost_matrix.shape[0] and col >= cost_matrix.shape[1]:\n",
    "                deaths.append(row)\n",
    "\n",
    "        ids_from = np.array(ids_from)\n",
    "        ids_to = np.array(ids_to)\n",
    "        births = np.array(births)\n",
    "        deaths = np.array(deaths)\n",
    "                        \n",
    "        # Account for +1 offset of the dense labels\n",
    "        ids_from += 1\n",
    "        ids_to += 1\n",
    "        births += 1\n",
    "        deaths += 1\n",
    "        \n",
    "        links = {\"links\": (ids_from, ids_to), \"births\": births, \"deaths\": deaths}\n",
    "        return links"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "697622ea-e7fd-4322-b538-a754106f2b0d",
   "metadata": {},
   "outputs": [],
   "source": [
    "bm_linker = BipartiteMatchingLinker(threshold=50, drift=(-20, 0), birth_cost_factor=1.05, death_cost_factor=1.05)\n",
    "bm_links = bm_linker.link(detections)\n",
    "bm_tracks = bm_linker.relabel_detections(detections, bm_links)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1e92ec76-0bda-4565-9c3b-34ff43c99ea6",
   "metadata": {},
   "outputs": [],
   "source": [
    "viewer = napari.viewer.current_viewer()\n",
    "if viewer:\n",
    "    viewer.close()\n",
    "viewer = napari.Viewer()\n",
    "viewer.add_image(x)\n",
    "visualize_tracks(viewer, bm_tracks, name=\"bm\");"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "cb377f3a-da64-45ec-9b34-e80a9a4508e0",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5d8bcfba-9be8-4699-8c6c-5ba6cfd70bd6",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "jupytext": {
   "formats": "ipynb,py:percent"
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined